# Agente Auto


Agente auto

In [471]:
# Model design
import agentpy as ap
import networkx as nx 
import random 

# Visualization
import matplotlib.pyplot as plt 
import seaborn as sns
import IPython

## About the model 
2 carriles en sentidos opuestos, 



## Defining the model

In [545]:
class Car(ap.Agent):
    
    def setup(self):  
        """ Initialize a new variable at agent creation. """
        self.condition = 0  # Conduciendo = 0, Standby = 1
        self.turn_chance = self.model.random
        self.grid = self.model.grid
        self.dir = 0 #Horizontal = 0, Vertical = 1
        self.side = 0 #Right = 0, Left = 1
        
        
        
    def drive(self):
        """ Manejar """
        ### Setters iniciales
        if (self.grid.positions[self] == (0, int(self.p.gridsize/2))):
            self.dir = 1
        elif (self.grid.positions[self] == (0, int(self.p.gridsize/2+1))):
            self.dir = 1
            self.side = 1
        elif (self.grid.positions[self] == ( int(self.p.gridsize/2), 0)):
            self.side = 1

        ### Lado de la calle
        if self.side == 0: #Right      
            if self.condition == 0 and self.dir == 0:  #Horizontal
                self.grid.move_by(self, (0,1))
            elif self.condition == 0 and self.dir == 1: #Vertical
                self.grid.move_by(self, (1,0))
        else: #Left
            if self.condition == 0 and self.dir == 0:  #Horizontal
                self.grid.move_by(self, (0,-1))
            elif self.condition == 0 and self.dir == 1: #Vertical
                self.grid.move_by(self, (-1,0))
                
        ### Girar
        rng = self.model.random
        if ((self.grid.positions[self] == (self.p.gridsize/2,self.p.gridsize/2)
             and self.dir == 1 and self.side == 0)and  # street is Turnable (V-R)
            self.p.turn_chance > rng.random()):
            self.dir = 0
            self.side = 1
        elif ((self.grid.positions[self] == (self.p.gridsize/2,self.p.gridsize/2+1)
             and self.dir == 0 and self.side == 1)and  # street is Turnable (H-L)
            self.p.turn_chance > rng.random()):
            self.dir = 1
        elif ((self.grid.positions[self] == (self.p.gridsize/2+1,self.p.gridsize/2)
             and self.dir == 0 and self.side == 0) and  # street is Turnable (H-R)
            self.p.turn_chance > rng.random()):
            self.dir = 1
        elif ((self.grid.positions[self] == (self.p.gridsize/2+1,self.p.gridsize/2+1)
             and self.dir == 1 and self.side == 1)and  # street is Turnable (V-L)
            self.p.turn_chance > rng.random()):
            self.dir = 0
            self.side = 0
    

        
                
        
       

        

In [546]:
class CarModel(ap.Model):
    
    def setup(self):
        rng = self.model.random
        """ Initialize the agents and network of the model. """
        self.grid = ap.Grid(self, (self.p.gridsize, self.p.gridsize), torus= True, track_empty= True)
        self.agents = ap.AgentList(self, 1, Car)
        if 0.5 > rng.random():
            self.grid.add_agents(self.agents, [(int(self.p.gridsize/2),0)])
        else:
            self.grid.add_agents(self.agents, [(int(self.p.gridsize/2+1),0)])
        
        
    def step(self):   
        """ Define the models' events per simulation step. """
        self.agents.drive()
        rng = self.model.random
        if self.p.car_chance > rng.random() and len(self.agents) < self.p.cars: 
            self.new_agents = ap.AgentList(self, 1, Car)
            self.agents.extend(self.new_agents)
            if 0.5 > rng.random():
                if 0.5 > rng.random():
                    self.grid.add_agents(self.new_agents, [(int(self.p.gridsize/2),0)], empty=True)
                else:
                    self.grid.add_agents(self.new_agents, [(int(self.p.gridsize/2+1),0)], empty=True)
            else:
                if 0.5 > rng.random():
                    self.grid.add_agents(self.new_agents, [(0, int(self.p.gridsize/2))], empty=True)
                    
                else:
                    self.grid.add_agents(self.new_agents, [(0, int(self.p.gridsize/2+1))], empty=True)
                    
                
        

    
        

## Running a simulation

In [547]:
parameters = {  
    'gridsize': 24,
    'cars': 9,
    'steps': 200,
    'turn_chance': 0.5,
    'car_chance': 0.1,
}

model = CarModel(parameters)
results = model.run() 

Completed: 200 steps
Run time: 0:00:00.026535
Simulation finished


## Analyzing results

In [548]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 5 keys
'reporters': DataFrame with 1 variable and 1 row
}

## Creating an animation

In [549]:
def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('condition')
    ap.gridplot(attr_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Car model \n Time-step: {model.t}")

fig, ax = plt.subplots()
model = CarModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())